# County NAICS Files
After running the notebook, delete the python/data_raw folder<br>
Zip level not available -> https://api.census.gov/data/2011/cbp/variables.html<br>
This notebook is a filtered and edited version of us_econ.ipynb for generating yearly data.<br>
For earlier code that generates averaged data aver years please refer us_econ.ipynb.<br>
This notebook will start generating data from year 2017.



In [ ]:
import csv
import requests as r
import pandas as pd
import zipfile, io
import os
from tqdm import tqdm
import pathlib
import pandas as pd
import datetime
endyear = datetime.date.today().year
api_headers = {}
api_headers['x-api-key'] = '975f39a54e48438ceebf303d6018e34db212e804'

In [ ]:
# Set a relative location to save the data from the request
repo_dir = pathlib.Path().cwd()
#print(repo_dir)

raw_data_dir = repo_dir / 'data_raw'
out_data_dir = raw_data_dir / 'BEA_Industry_Factors'
    
county_data_dir = out_data_dir / 'county_level'
if not county_data_dir.exists():
    county_data_dir.mkdir(parents=True)
